In [16]:
import json
import numpy as np
import pandas as pd

import os
from sklearn.externals import joblib


def init(model_path="model.pkl"):
    global model
    model = joblib.load(model_path)

def predict(data):
    try:
        #data = np.array(json.loads(data))
        result = model.predict(data)
        # You can return any data type, as long as it is JSON serializable.
        return result
    except Exception as e:
        error = str(e)
        return error


def predict_proba(data):
    try:
        #data = np.array(json.loads(data))
        result = model.predict_proba(data)
        # You can return any data type, as long as it is JSON serializable.
        return result
    except Exception as e:
        error = str(e)
        return error


In [17]:
id_name = 'id'
target = 'ACTION'
model_path = "d_aea_model.pkl"
test_file = "test_d_aea.csv"

init(model_path)

test_df = pd.read_csv(test_file)
y_true = test_df[target]

y_pred = predict(test_df.drop([target], axis=1))

print("accuracy:")
print(np.mean(y_pred==y_true))

from sklearn import metrics

y_pred_prob = predict_proba(test_df.drop([target], axis=1))

fpr, tpr, thresholds = metrics.roc_curve(y_true, y_pred_prob[:,1], pos_label=1)
print("auc:")
print(metrics.auc(fpr, tpr))

accuracy:
0.9734513274336283
auc:
0.9316193036596679


In [28]:
## predcict test dataset on Kaggle

kaggle_submit_head = ["Id", "Action"]

kaggle_test_file = "kaggle-test-d_aea.csv"
kaggle_pred_file = "kaggle-test-d_aea-predictions.csv"

kaggle_test_df = pd.read_csv(kaggle_test_file)
kaggle_test_id = np.array(kaggle_test_df[id_name])
#kaggle_y_true = np.array(kaggle_test_df[target])

print(kaggle_test_df.head())
print(kaggle_test_df.shape)

kaggle_y_pred = predict(kaggle_test_df.drop([id_name], axis=1))

print(kaggle_y_pred.shape)

   id  RESOURCE  MGR_ID  ROLE_ROLLUP_1  ROLE_ROLLUP_2  ROLE_DEPTNAME  \
0   1     78766   72734         118079         118080         117878   
1   2     40644    4378         117961         118327         118507   
2   3     75443    2395         117961         118300         119488   
3   4     43219   19986         117961         118225         118403   
4   5     42093   50015         117961         118343         119598   

   ROLE_TITLE  ROLE_FAMILY_DESC  ROLE_FAMILY  ROLE_CODE  
0      117879            118177        19721     117880  
1      118863            122008       118398     118865  
2      118172            301534       249618     118175  
3      120773            136187       118960     120774  
4      118422            300136       118424     118425  
(58921, 10)
(58921,)


In [27]:
print(kaggle_test_id.shape)

assert kaggle_y_pred.shape == kaggle_test_id.shape

kaggle_pred_df = pd.DataFrame({kaggle_submit_head[0]: kaggle_test_id, kaggle_submit_head[1]: kaggle_y_pred})

print(kaggle_pred_df.head())


kaggle_pred_df.to_csv(kaggle_pred_file, index=False)

(58921,)
   Id  Action
0   1       1
1   2       1
2   3       1
3   4       1
4   5       1
